# Train NER

In [ ]:
!pip install -q datasets transformers
import re
import pandas as pd

     |████████████████████████████████| 451 kB 4.3 MB/s 
     |████████████████████████████████| 5.8 MB 36.9 MB/s 
     |████████████████████████████████| 212 kB 64.0 MB/s 
     |████████████████████████████████| 132 kB 61.4 MB/s 
     |████████████████████████████████| 182 kB 54.3 MB/s 
     |████████████████████████████████| 127 kB 31.2 MB/s 
     |████████████████████████████████| 7.6 MB 50.6 MB/s 


In [ ]:
def get_tokens_with_entities(raw_text: str):
    raw_tokens = re.split(r"\s(?![^\[]*\])", raw_text)

    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags=re.I|re.M)

    tokens_with_entities = []

    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token)
        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")

            for i, raw_entity_token in enumerate(re.split("\s", raw_entity_value)):
                entity_prefix = "B" if i == 0 else "I"
                entity_name = f"{entity_prefix}-{raw_entity_name}"
                tokens_with_entities.append((raw_entity_token, entity_name))
        else:
            tokens_with_entities.append((raw_token, "O"))

    return tokens_with_entities

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Implement NER data maker class for create tuples

In [ ]:
class NERDataMaker:
    def __init__(self, texts):
        self.unique_entities = []
        self.processed_texts = []

        temp_processed_texts = []
        for text in texts:
            tokens_with_entities = get_tokens_with_entities(text)
            for _, ent in tokens_with_entities:
                if ent not in self.unique_entities:
                    self.unique_entities.append(ent)
            temp_processed_texts.append(tokens_with_entities)

        self.unique_entities.sort(key=lambda ent: ent if ent != "O" else "")

        for tokens_with_entities in temp_processed_texts:
            self.processed_texts.append([(t, self.unique_entities.index(ent)) for t, ent in tokens_with_entities])

    @property
    def id2label(self):
        return dict(enumerate(self.unique_entities))

    @property
    def label2id(self):
        return {v:k for k, v in self.id2label.items()}

    def __len__(self):
        return len(self.processed_texts)

    def __getitem__(self, idx):
        def _process_tokens_for_one_text(id, tokens_with_encoded_entities):
            ner_tags = []
            tokens = []
            for t, ent in tokens_with_encoded_entities:
                ner_tags.append(ent)
                tokens.append(t)

            return {
                "id": id,
                "ner_tags": ner_tags,
                "tokens": tokens
            }

        tokens_with_encoded_entities = self.processed_texts[idx]
        if isinstance(idx, int):
            return _process_tokens_for_one_text(idx, tokens_with_encoded_entities)
        else:
            return [_process_tokens_for_one_text(i+idx.start, tee) for i, tee in enumerate(tokens_with_encoded_entities)]

    def as_hf_dataset(self, tokenizer):
        from datasets import Dataset, Features, Value, ClassLabel, Sequence
        def tokenize_and_align_labels(examples):
            tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

            labels = []
            for i, label in enumerate(examples[f"ner_tags"]):
                word_ids = tokenized_inputs.word_ids(batch_index=i)
                previous_word_idx = None
                label_ids = []
                for word_idx in word_ids:
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:
                        label_ids.append(label[word_idx])
                    else:
                        label_ids.append(-100)
                    previous_word_idx = word_idx
                labels.append(label_ids)

            tokenized_inputs["labels"] = labels
            return tokenized_inputs

        ids, ner_tags, tokens = [], [], []
        for i, pt in enumerate(self.processed_texts):
            ids.append(i)
            pt_tokens,pt_tags = list(zip(*pt))
            ner_tags.append(pt_tags)
            tokens.append(pt_tokens)
        data = {
            "id": ids,
            "ner_tags": ner_tags,
            "tokens": tokens
        }
        features = Features({
            "tokens": Sequence(Value("string")),
            "ner_tags": Sequence(ClassLabel(names=dm.unique_entities)),
            "id": Value("int32")
        })
        ds = Dataset.from_dict(data, features)
        tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)
        return tokenized_ds

## Remove_stopwords function

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
  stop_words = set(stopwords.words('english'))

  word_tokens = word_tokenize(text)

  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  for w in word_tokens:
    if w not in stop_words:
      filtered_sentence.append(w)


  res = " ".join(filtered_sentence)
  res = re.sub(r'\[\s+', r'[',res)
  res = re.sub(r'\s+\]', r']',res)
  res = re.sub(r'\(\s+', r'(',res)
  res = re.sub(r'\s+\)', r')',res)
  res = re.sub(r'\]\s+\(', r'](',res)
  return res

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Train Data

In [ ]:
raw_text = """
education. course. language experience. [university of science and culture](university). ph. [management](skill) current. [tourism management](skill). [university of payam noor](university). ba. [tourism management](skill). [front end developer](job_title). coding frond. [product design](skill). [digital photography](skill). [writing reading english](skill). listening. speaking. listening. speaking fatemeh farah ani. hobbies. traveling. reading. [photograph](skill) [product design](skill). farah ani. [figma](skill). [zeplin](skill). [javascript](skill). [html](skill) [css](skill). [microsoft office](skill) [product design](skill). crypto eye tehran jan current. [ux researcher](job_title) part time. [user experience consulting](job_title) part time. bourse yet eh ran may jan. [ux researcher](job_title). [content manager](job_title) [ux researcher](job_title). [head of service quality](job_title). skills. portfolio|
linkedin. at aba ki. portfolio. be hance. tab aki. skills. [design thinking](skill). [user centered design](skill). [user research](skill). [competitive analysis](skill). [thematic analysis](skill). [user persona](skill). [customer journey](skill) map. [information architecture](skill). [user flow](skill). [wireframe](skill). [visual design](skill). [interaction design](skill). [color psychology](skill). [ui design](skill) pattern. [material design](skill). [prototype](skill). [usability testing](skill). familiar with [agile](skill). tools. [figma](skill). [adobe xd](skill). [adobe illustrator](skill). [adobe photoshop](skill). [adobe light room](skill). [zeplin](skill). intermediate [html](skill) [css](skill). [javascript](skill). language. [ux ui designer](job_title). it ry to finds imp le. experience. [ux ui designer](job_title). documentation in progress. design [interactive prototype](skill). [ux ui designer](job_title). design [user flow](skill) ina team. ate amin [figma](skill). [ux ui designer](job_title). design [mobile ui](skill) kit. responsive mobile version ina team. dashboard in [figma](skill). [ux designer](job_title). dashboard in [figma](skill). [ui designer](job_title). design [mobile ui](skill) kit. [ux ui intern](job_title). dux lab academy jan may. design [interactive prototype](skill). education. [interaction design foundation](university). [ux ui design](skill) present|
linkedin. at aba ki. portfolio. be hance. tab aki. skills. [design thinking](skill). [user centered design](skill). [user research](skill). [competitive analysis](skill). [thematic analysis](skill). [user persona](skill). [customer journey](skill) map. [information architecture](skill). [user flow](skill). [wireframe](skill). [visual design](skill). [interaction design](skill). [color psychology](skill). [ui design](skill) pattern. [material design](skill). [prototype](skill). [usability testing](skill). familiar with [agile](skill). tools. [figma](skill). [adobe xd](skill). [adobe illustrator](skill). [adobe photoshop](skill). [adobe light room](skill). [zeplin](skill). intermediate [html](skill) [css](skill). [javascript](skill). language. [ux ui designer](job_title). it ry to finds imp le. experience. [ux ui designer](job_title). create [low fidelity wireframes](skill) in [figma](skill). documentation in progress. design [interactive prototype](skill). [ux ui designer](job_title). design [user flow](skill) ina team. ate amin [figma](skill). [ux ui designer](job_title). design [mobile ui](skill) kit. responsive mobile version ina team. dashboard in [figma](skill). [ux designer](job_title). dashboard in [figma](skill). [ui designer](job_title). design [mobile ui](skill) kit. [ux ui intern](job_title). dux lab academy jan may. design [interactive prototype](skill). education. [interaction design foundation](university). [ux ui design](skill) present|
expert. mid. lows kills. energetic [ui](skill). [xd](skill). have completed several projects. professional traveler there. online services. other easier. roya stour agency website. open banking payment banks. san bod holding timed ur at ion mos. specially online payment. by using this app you. can predict breast cancer more than. interesting. and now users. another challenge was input data. experiences. education. entertainments portfolio mohamad dad mand. [ui ux designer](job_title). dad mand mail|
baba pour. product features. would improve [user experience](skill). [product designer](job_title). [product design intern](job_title). education. [user research](skill). [ui ux design](skill). [product design](skill). email. linkedin. reliable and. resourceful approach. work history. excellent excellent. excellent. excellent excellent [product designer](job_title). company is fah an. diploma [ux](skill) fundamentals course. [ux](skill) [ui](skill) inaction course. [urmia university](university). certifications. io|
contact ars ham. [design](skill) award. res oxy [design](skill) process. products tot he client. skills|
combe hance. thinking. sc. efficiently. [teamwork](skill). selection to final payment. [teamwork](skill). javan ham ta. present. skills. currently ins ketching. stage|
erf an rost ami yan. [product design](skill). at. selfs tu di. associate information technology. personal information. years. age. male. gender. tehran. location. single. marital status. completed. military services tatus. contact details. phone. mobile. email. address. page link. linkedin. years. work experience. tomans. salary expectation. job vision. last update. reference id. work experience. education. languages. english. upper intermediate. software. [adobe xd](skill). advanced. [sketch](skill). advanced. [adobe illustrator](skill). intermediate. [adobe photoshop](skill). intermediate. [figma](skill). intermediate. [zeplin](skill). intermediate. additional skills. [wireframe](skill). [user journey](skill) map. persona. [user flow](skill). [design thinking](skill). [visual design](skill). [user flow](skill) journey map. [interaction design](skill). about me. until now. months. [product design](skill). selfs tu di. iran tehran. months. [product designer](job_title). iran tehran. months. [ui ux designer](job_title). pars rad in. iran tehran. until april. months. [ui ux designer](job_title). iran tehran. associate information technology|
coms kills. hobbies about. experiences. self employed monta video game. [teamwork](skill) iso ne of. [figma](skill). [sketch](skill). [adobe xd](skill). [balsamiq](skill). [illustrator](skill). [visual design](skill). [ux research](skill). [design thinking](skill). [ui designer](job_title). december now. experience. thete am tod oo urb est. en nhan cement. player application and movies website. phone. airbnb colors ands ty les. whitespace was an. phone. good accessibility anda modern ui. which we had in thete am. desktops ee full images. phone. monta. [concept design](skill) applications redesign. see full images on. mask an bank. ham rahman application redesign. application redesign. mar. sound cloud. [concept design](skill) applications redesign. see full images on. shoes app. music player [application design](skill). [application design](skill). mar|
about me. click to view my portfolio. work experiences. conducting stakeholder reviews presentations. desktop version. tehran iran. may june. users. tehran iran instagram. [ui](skill) [ux](skill). youtube. linkedin. skills. [ux design](skill). [ui design](skill). [product design](skill). [user centered design](skill). [strong communications](skill). [design thinking](skill). [user journey](skill) map. [user flow](skill). [information architecture](skill). [wireframing](skill) [sketching](skill). component library style guide. [interaction design](skill). [usability testing](skill). using psychology principles in [ux](skill). [agile scrum](skill) framework. contract [ui ux designer](job_title). online shopping website platforms. education. language. [adobe xd](skill). [figma](skill). [adobe photoshop](skill). [google analytics](skill). other activities. [freelance ui ux designer](job_title). in bulk to buyers. sellers|
alireza khan in ej ad. [graphics](skill). [ui ux](skill). business successful. education. samp ad. ba [industrial design](skill). dig izar gar. [ui](skill) [ux](skill). [sketching](skill). [product design](skill). [rendering](skill). [market research](skill). [strategy](skill) service innovation. skills applications. english skills. speaking. listening. writing. reading ostrich. [senior designer](job_title) [graphics](skill). manal isa art gallery. mm. mm mm. mm mm. mm. mani books planner [design](skill). [ui ux design](skill). [ui ux design](skill). [ui ux design](skill). dig izar ar website design. [ui ux design](skill). [ui ux design](skill). dat [website design](skill) concept. [product design](skill). largo. [product design](skill). [product design](skill). [product design](skill). others|
[user research](skill) methods. observation. interview. [surveys and questionnaires](skill). [competitive analysis](skill) focus groups. scenarios. [user flow](skill) [user journey](skill). [evaluation methods](skill). [usability test](skill). [cognitive walk through](skill). [sketching](skill). [wireframing](skill). [draw](skill). [ux designer](job_title). google. rout aa|
[ux designers](job_title) or ou sh. comport folio link|
computers oft ware. province. [ui ux designer](job_title). everyday. contact info. education. tools. course. [figma](skill). [html](skill). ideate. [user flow](skill). [wireframe](skill) [persona](skill). [ux research](skill). [usability test](skill). [adobe xd](skill) [zeplin](skill). [maze](skill) instagram. [bootstrap](skill). [javascript](skill). linkedin. be hance|
mor ova ti. [product designer](job_title). profile. statement. education. work. experience. certificates. and. master of complex systems. bachelor of physics. [university of tehran](university). rental website. fora rental house website. builder. restaurants andre tails from step one. [ui ux consulting](job_title). designing newsletters layout. online store. january. january. [prototyping](skill) [interaction design](skill) [python](skill) [git](skill). [user research](skill) [data driven design](skill). development [university of tehran](university). honorary member of independent charity. languages. volunteer. activities. interests|
social channels [ux designer](job_title). [interaction design](skill). learning freelancer. freelancer. based on [ux](skill) principles. shots available on [figma](skill). operators. their business on rout aa map. sales profit. website ia. developed version. series. shots available on. [figma](skill) [ux ui designer](job_title). [ux ui designer](job_title). matin momentum gears media port. comb as edin ire land. food catering online platform. touser experience|
[research](skill) empathy. [wireframes](skill). foundations. [html](skill) [css](skill). [java](skill) minor. working with web flow. web interactions. [architecture](skill). bachelors degree. [architecture](skill). masters degree. [figma](skill). [illustrator](skill). [photoshop](skill). [aftereffects](skill). [rhino](skill) gas shopper. keys hot [redshift](skill). [spline](skill). [design](skill) learned. advanced [ui ux design](skill). [ux design](skill). [web design](skill). no code [web design](skill). web flow online university. education. [azad university tehran central branch](university). entry exam. [university of tehran](university). skills. [ui design](skill). [graphics](skill) presentation. experiences. others. responsible perseverance precise and creative. coma mir pashas ad ehs. web flow. io. web flow|
alireza. fate mia lire za fate mi. for in my future job. people psychologically. courser. [journey map](skill) pin. media pages. [ux process](skill). [user flow](skill) [persona](skill). empathy map [journy map](skill). [ux research](skill). [interview usability](skill). study contextual. [data visualization](skill). education projects. courses skills|
page of my works. [product designer](job_title). work experience. [design](skill) [user flow](skill). [design](skill) [wireframes](skill). [design color palette](skill). making components in [figma](skill). making [prototypes](skill). page of courses. languages. native persian. september september. [figma](skill). [google analytics](skill) [balsamiq](skill) [mockup](skill). [clarity](skill) [adobe illustrators](skill) kills. [user research](skill) [wireframing](skill). [illustration](skill)|
tehran county tehran iran. linkedin. be hance. summary. experience. [product designer](job_title). of stock market. [ui ux designer](job_title). address click. my responsibilities in this company. mobile and web panels. [product designer](job_title). fe ba ug year months. [product designer](job_title) [product owner](job_title). defining product features. [business developer](job_title) [ux designer](job_title). zo ha kish telecom iran. [maya](skill) ug years months. education. skills|
bahar. about me portfolio. contact me. education. interests. article company work experience. marm ar [ux designer](job_title). with its features. tests. freelance work experience. essential. jun. places at ag lance. skills. [ux](skill). [gamification](skill). [project management](skill). tools. [adobe illustrator](skill). bahar. comp lease visit my website. [design](skill). is fah ani ran. be hance. comb aw hari. [software engineering](skill). native. english. limited working proficiency. cycling technology arts psychology. animals painting|
education. work experience. petrochemical products. technology. summer. equations. skills. [python](skill) [matlab](skill). [statistics](skill) and [mathematics](skill) [sql](skill). [arduino](skill) expert choice. languages kills. english fluent. others kills. [self motivation](skill) [time management](skill) skills. license certificate. honors anda wards. schools among participants in iran. among participants in iran. publications|
fast learner. challenges in my job. flexible when it comes tote am work. reading books. sports. device. creates efficient ais er vices. passionate about learning new concepts. andi love new challenges in my work. because challenges make life exciting. phone career overview. gran far. [machine learning engineer](job_title) works umma ry. [algorithms](skill) such as [bert](skill). working on [text to speech](skill) project. ford at aga the ring. per sian book translation. [alzahra university](university). projects. bachelor of science. technical knowledge. advanced [python programming](skill). [hugging face](skill). [beautifulsoup](skill) eleni um. familiarity with [docker](skill)|
curriculum vitae. personal information. gender female. nationality iran ian. phone. education. ph. bi rj andi ran. technicals kills and experience. nonparametric estimation. [probability](skill). [survival analysis](skill). [machine learning](skill). [programming](skill) language. work experiences. investment of insurance companies. [teaching](skill). technology iran. mar. [university of guilan](university) iran. experience with [clinical research](skill). address. gu ilan iran tel fax email. [teaching assistant](job_title). jan. fe bj un. [research](skill) interests. [survival analysis](skill). [actuarial science](skill). [machine learning](skill). [nonparametric statistics](skill). [probability](skill). publication in reverse chronological order. published papers. learning techniques. journal. pone. asymptotic properties oft he. estimator of cumulative residual [entropy](skill). [statistics](skill). two parameter [exponential distribution](skill). journal of [statistical theory](skill) and. jo mho oris. [nonparametric sampling](skill). submitted papers. some asymptotic properties of kernel. review. conferences. jo mho oris. kernel type estimator for. jo mho oris. stochastic orders based on. applications tehran iran. jo mho oris. percentile residual life and. presented ins tat is tical. conference in. [university of science technology](university) iran. hobbies|
tehran iran. live tas dig hi. [github](skill). linkedin. tas dig hi. stack overflow. comus ers. skills. [mathematics](skill) [statistics](skill). [tensorflow](skill) [kera](skill) spy [torch](skill). [unit test](skill) [oop](skill) [solid](skill) [design patterns](skill). [agile scrum](skill) [kanban](skill). [git](skill). education. master of science [artificial intelligence](skill). thesis [image encryption](skill) based on. and chaos game. bachelor of science computer. thesis online handwriting. graphology [feature extraction](skill) and. identification about me. students olympiad. development and creating predictive models. ia man. attitude. intelligence fields. work experience. accuracy. stack [python](skill) [mongo](skill). [strategic decisions](skill). may present. certificates. [neural network](skill) and deep. languages. english fluent. de uts ch germany beginner. interests. painting. watching movies. yoga. methods. [math olympiad teacher](job_title) math house. knowledge|
fatemeh. nika nyah oo. tehran iran. education. [university of isfahan](university). is fah ani ran. master of science [computer engineering](skill). shah rood iran. recognition. projects. paper. [matlab](skill). recognition thesis. [matlab](skill). simple [text mining](skill). [python](skill). [python](skill). [python](skill). work experience. [java programmer](job_title). present. [entrepreneurship](skill). greens hop. water environment and energy. innovation and technology center. group. preparation of reception packages. private [tutor](job_title). [mathematics](skill) [formal languages and automata](skill) [programming](skill). skills. [git](skill). [matlab](skill). [python](skill). [sqlite](skill). [image processing](skill). [machine learning](skill). [weka](skill). [photoshop](skill). [neural networks](skill). [text processing](skill). [java](skill). [latex](skill). achievements. among participating groups. certificates. [python](skill). languages. english. germany. interests. [programming](skill). [researcher](job_title). [computer vision](skill). [machine learning](skill). [deep learning](skill). [neural networks](skill). [image processing](skill). courses. achievements tasks|
become. interested. in. [programming](skill). and. started. with. [python](skill). from. the. middle. of. my. undergraduate. studies. and. then. started. learning. [machine learning](skill). and. [neural networks](skill). participated. in. the. masters. degree. entrance. exam. in. and. was. accepted. in. the. field. of. [computer engineering](skill). [artificial intelligence](skill). at. [shahid university](university). am. also. interested. in. [algorithms](skill). and. its. challenges. and. [teamwork](skill). bachelor in [electrical engineering](skill). branch control. master in [computer engineering](skill). branch [artificial intelligence](skill). tehran iran. since. [python](skill). [image processing](skill). [natural language processing](skill). [computer vision](skill). [deep learning](skill). [machine learning](skill). [tensorflow](skill). [django](skill). [web design](skill). [java](skill). reading level. writing level. speaking level. listening level. english. [artificial intelligence](skill). dob. marital status. single. military service. exemption. bag her ia sh kang mail. profiles umma ry. education. skills. language. link. deep data. link. deep data. [image captioning](skill). link. deep data. [video classification](skill) with [transformers](skill). link. deep data. [text classification](skill) with [transformer](skill). link. deep data. link. deep data. link. deep data. [sentiment polarity detection](skill). link. deep data. link. deep data. link. deep data. link. deep data. implementation of. link. deep data. link. deep data. link. deep data. ab dev. projects. [social network](skill)|
education. honors anda wards. present. [research](skill) sand academic projects. under. [programming](skill). [teaching](skill) experiences. skills. software. and [matlab](skill). personality. languages. employment history|
ala vine jad. work experience. to. [data scientist](job_title). [research](skill) on [online learning](skill). [data scientist](job_title). [student](job_title) job full time. [machine learning engineering](job_title). company asa lu yeh iran. purposefully. [machine learning engineering](job_title). asa lu yeh iran. suitable period of company sov er haul. education. sc. it. sc. sc. [tutor](job_title) of [linear control](skill) workshop. phone. email. language. per sian mother tongue. certifications. prof. of [stanford university](university). mach in learning online course. [linear regression](skill) logistic. [machine learning](skill) online course with. [python](skill). skill. [python](skill) language. [statistics](skill) mach in learning. [statistical analysis](skill). linear logistic [clustering](skill) [regression](skill)|
linkedin. summary. improve sales. experience. [data scientist](skill). self employed. freelancing. [deep learning](skill) [programming](skill). education. [university of tehran](university). present. amounts using [deep learning](skill). advisor mali he dane sh ph. licenses certifications. skills. honors awards|
page. [machine learning engineer](job_title). personal details. address. tehran pars tehran iran. phone. email. sky pe. hamed. linkedin. linkedin. dob. nationality. iran ian. languages. german as umma ry. commercial network [data analysis](skill). skill highlights. [artificial intelligence](skill). [decision making](skill) ability. [blockchain](skill). [feature engineering](skill). [social networks](skill). [reinforcement learning](skill). [data mining](skill). [statistical analysis](skill). work background. present. [teaching](job_title) online classes. [machine learning engineering](job_title) present. [data gathering](skill). developer in [artificial intelligence](skill) present. [reinforcement learning](skill). hoosh ex arf tehran. [technical analysis](skill). bani teja rat tehran. item recommendation. jewelry virtual try on. page. [programming](skill) languages. [python](skill). [matlab](skill). [java](skill). [php](skill). [databases](skill). [sql server](skill). [mysql](skill). packages. [pandas](skill). [matplotlib](skill). [tensorflow](skill). backend. [django](skill). [flask](skill). [asp net core](skill). other. [windows](skill). [ubuntu](skill). ser vises and [microservices](skill). [cluster computing](skill). [git](skill) [software consultant](job_title). [software development](skill). [research and development expert](job_title) to. [head of laboratory](job_title). [quality control](skill). hygienic fiber. test design. university [tutor](job_title) to. projects. webs er vices combination. data. tasks che dule rin cloud fog and. [clustering](skill) and [classification](skill). matching ontologies. detecting unconventional mobile applications. [machine learning](skill) [algorithm](skill) [optimization](skill). page. networks. churn customer prediction. [reinforcement learning](skill). [deep learning](skill) model tuning. target detection. [deep learning](skill). [text mining](skill) and [image mining](skill). [detecting image content](skill). [stock market price forecasting](skill). [signal processing](skill). [feature extraction](skill) from audio. diagnosis of micro aneurysm. vessel detection. education. certifications. [quality control](skill) for nonwoven. researches. [android](skill) project. [human resource](job_title) monitoring. [android](skill). [php](skill). personal health records. [android](skill). [php](skill). [django](skill) project. book recomendation. forecasting. gathering. page. journal. references available on request|
work experience. [research assistant](job_title). city tehran. country iran. [simulation](skill) league consists of agents. [algorithms](skill). projects. opponents. [teacher assistant](job_title). city tehran. country iran. [programming](skill) kills. [programming](skill) concepts. average rating. nationality iran ian. date of birth. gender female. email address maryam. address tehran iran. [teacher assistant](job_title). city tehran. country iran. [programming](skill) skills. [programming](skill) concepts. average rating. private [tutor](job_title). plat ute current. city tehran. country iran. [australia universities](university). [instructor](job_title). city tehran. country iran. students. internship. city tehran. country iran. internship. city tehran. country iran. [database](skill). [database](skill). internship. roshan company. city tehran. country iran. projects. [classification](skill). education and training. sc. [computer engineering](skill). west. tehran iran tehran iran. fundamentals of [reinforcement learning](skill). address alberta remote canada. ai fundamentals course. address tehran iran. english. mother tongues per sian. other languages. digitals kills. languages. frameworks. net xu nit [django](skill). tools. platforms|
[data scientist](job_title). linkedin. education. study program. bachelor of [software engineering](skill). personal projects. analysing nanci alda ta. models. recognizing german roadsign. [network](skill). present. model validation and parameter. using [python](skill) [natural language toolkit](skill). present. work experience. [animator](job_title). workplace company. [web developer](job_title). [data scientist](job_title). present. skills. [python](skill) [programming](skill). [natural language processing](skill). [sentiment analysis](skill). semantics ear ch. languages. english. interests. book. climbing. nights ky. math. animation. traditional music. learning|
link. tehran. experience. tehran. recognition. per sian. sentences. from. arabic. [python programmer](job_title). khors hid soft. tehran. jan. keyword. extraction. text. [analysis](skill). image. animation. deep. fake. convert. words. to. per sian. [scrape data](skill). for. news and hotels. tehran. [machine learning researcher](job_title). intern. education. maleka sh tart eh rani ran. thesis title. learning. article. semantic. similarity. of. per sian. sentences. using. vector. space. adaptation and [deep learning](skill). skills. concepts. knowledge of [ai](skill). [machine learning](skill). and [deep learning](skill). embedding concepts. and libraries. [tensorflow](skill) gate. [topic modeling](skill). familiarity with bert. [transformer](skill) elmo. familiar in [git](skill). familiar in [linux](skill). languages. per sian english|
bag her ia ut. phone citizenship iran. ahmad nick aba di. [research](skill) interests [artificial neural networks](skill). [computer vision](skill). [deep learning](skill). sc. of [computer engineering](skill). sit yen trance exam forb. sc. [research](skill). works. [teaching](skill). ahmad nick aba di. hoss ein zeina li. sara bo urb our. [teacher assistant](job_title) fall. mahmoud mom taz pour. industry. [web programmer](job_title) fall. skills [programming](skill). [python](skill) [java](skill). libraries. languages. english intermediate. sc. [machine learning](skill). [statistical pattern recognition](skill). [statistical language processing](skill). [artificial neural networks](skill). [probabilistic graphical models](skill). [statistical machine learning](skill). [evolutionary computing](skill). [machine vision](skill). nick aba dia ut|
alireza shah edi. st. tehran iran. emails hah edi. ali ut. linkedin page. education. fundamentals of [reinforcement learning](skill). [data science](skill) professional. data camp. data camp. [image processing](skill) with [keras](skill) in [python](skill). data camp. of. [neural networks](skill) and [deep learning](skill). [deep learning](skill). journal publications. jo lai. he di. advanced [manufacturing technology](skill). current work. conference publications. tomer satisfaction. mans hah. abi objective model. ron mental factors. of. application of de ma. anew facility location model for. english. [research](skill) interests. [autonomous systems](skill). remanufacturing masters project. bachelors project. of. bachelors project. bachelors project. softwares kills. [programming](skill) languages. [solidworks](skill) [catia](skill) auto desk alias. miscellaneous. of|
personal information. telephone number. education. [university of tehran](university) present. master ina. [university of tehran](university). sc in [electrical engineering](skill). diplomas in physics mathematics. [research](skill) interests. [data science](skill). honors awards. sc. [research](skill) experience. social [reinforcement learning](skill). sales problem with imbalanced data. solving decentralized [optimization](skill) problem. [teaching](skill). [teaching assistant](job_title). skills. [programming](skill) languages. engineerings oft ware. [git](skill) [matlab](skill) [arm programming](skill). [operating system](skill) and environment. [linux](skill) [windows](skill). generals oft ware. languages. listening. reading. speaking. writing. professional experience. [arm programmer](job_title). interests. sports reading books arts|
for. implemented in [fpga](skill). too. systems. oe ducat ion. [arak university](university) arak iran ph. [geometry](skill). sc. in. [differential geometry](skill). minimizing. in [mathematics](skill). [mathematical modeling](skill). in [linux](skill). [embedded programming](skill). [git](skill) doxy gen [docker](skill). withe. bull. argentina. withe. scient ia rum [mathematic](skill) arum. withe. informatics. withe. [nonlinear](skill) anal. journal of. [mathematical physics](skill). virtual venue. iran. applications may malay ari ran. ann. iran ian math. conference july tab riz iran. july bona biran. winter. university in iran from to. to. constantina rc us yahoo|
arminda sh ti. tehran. about me. skills. [financial market](skill) [ubuntu](skill). experience. [technical support](job_title). self employed. national iran ian oil company. [software engineer](job_title). self employed. present. [research](skill). data set. accuracy. scale. data set. train. train. data set. test. [tensorflow](skill)|
education. diploma in physics and [mathematics](skill). honors anda wards. experience. mathematical olympiad. courses. [machine learning](skill). [data structures](skill). [linear algebra](skill). [statistics](skill) and [probability](skill). projects. technical skills. frameworks [github](skill) [git](skill)|
personal data. date of birth april. phone. email mahdi. [github](skill). io. [research](skill) interest. [information retrieval](skill) [machine learning](skill). [natural language processing](skill). education. shape. based onus ers information need. moreover. query embedding. it is. ahmad alia bin. [golestan university](university) gorgan iran. publications. asha pea ware framework. information processing management journal. user grouping method. the unfairness of. per. in proceedings. management no. temporal expert proling with. information processing management no. novel [clustering](skill) based approach. term mismatch problem. translation model. [teaching assistant](job_title) ship. honors anda wards. in computers oft ware. skills. [database](skill) [mysql](skill) [sql](skill). [lucene](skill). work experience. mar. mar. june. industrial projects. tweet. main methodology [parsbert](skill). mar. mar. main methodology [fasttext](skill). interesting course projects. selected course. academics er vices. languages. english upper intermediate|
tehran iran. since. tehran iran. since. tehran iran. mobile. phone. marital status single. [data scientist](job_title). profile summary. education. skills. language. [machine learning](skill). [social network](skill). [machine learning](skill). [stanford university](university). equivalent toon campus courses. participation in the course|
[data scientist](job_title). tehran. phone email. matin. [github](skill). linkedin. education. tehran. iran. thesis. skills. technical generals kills. advanced. [python](skill) [cython](skill) [java](skill). [linux command line](skill) [git](skill). [relational databases](skill). [data cleaning](skill) [data visualization](skill). [research](skill). intermediate. [docker](skill). theoretical skills. advanced. [linear algebra](skill). [probability](skill) [statistics](skill). [deep learning](skill). [reinforcement learning](skill). [optimization](skill). [deep learning](skill) specialization. course. [neural networks](skill) and [deep learning](skill). course. tuning. [regularization](skill) and [optimization](skill). course. structuring [machine learning](skill) projects. course. [convolutional neural networks](skill). course. sequence models. introduction to [numerical analysis](skill). [data scientist](job_title) with [python](skill) data camp. intermediate [python](skill). [python](skill) [data science](skill) toolbox part. [python](skill) [data science](skill) toolbox part. [pandas](skill) foundation. intermediate importing data in [python](skill). cleaning data in [python](skill). manipulating data frames with [pandas](skill). merging data frames in [pandas](skill). completed. in progress. [convex optimization](skill) edx. work experience. august december. [teaching](skill) experience. [linear algebra](skill). [teaching assistant](job_title) bi he. [teaching assistant](job_title) bi he. [teaching assistant](job_title). bi he. [teaching assistant](job_title). bi he|
sc [student](job_title) of it. saeed. tehran iran. lives ae ed. [github](skill). familiar with. [python](skill). [rapid miner](skill). [machine learning](skill). [git](skill). [docker](skill). languages. english. interests. [data mining](skill). [machine learning](skill). [deep learning](skill). education. bachelor in [electrical engineering](skill). present. uni projects. classic at ion. understood. [clustering](skill). similar objects in. the same group. we use [clustering](skill) method in. [python](skill) to [cluster](skill). [regression](skill) and [outlier detection](skill). we use [regression](skill). combined cycle power plant data set. [object detection](skill). itself. we use. books and courses. hands on|
mohamad reza sheikh. linkedin. summary. my [github](skill) repository. experience. [python developer](job_title). founder. education. diploma of education [mathematics](skill). licenses certifications. mohamad reza sheikh page. skills. mohamad reza sheikh page|
shadi tavakoli. [data scientist](job_title). shadi. tava ko lig mail. tehran iran. linkedin. work experience. [data scientist](job_title). telewebion. present. tehran. telewebion [speech recognition](skill). [machine learning engineer](job_title) intern. han oop. tehran. biotech company. images. in. segmentation. heart. classic at ion of heart disease. education. present. [machine learning](skill). [reinforcement learning](skill). [image processing](skill). skills. [python](skill). [machine learning](skill) concepts and. [algorithms](skill). [computer vision](skill). [statistics](skill). [tensorflow](skill). personal projects. recognition. emotion. face. captioning. image. achievements. iran. april. open. iran. participation. of. robot. de miner. intelligent. size. small. competitions. specialization. learning. deep. languages. english. french. achievements tasks. achievements tasks. courses|
[data scientist](job_title). profiles umma ry. hi. fard ad. [student](job_title). of. [computer science](skill). have. successfully. passed. the. courses. of. advanced. [python](skill). [programming](skill). and. [machine learning](skill). with. master. and. at. this. moment. work. on. my. personal. projects. dob. marital status. single. military service. educational exemption. tehran iran. education. ka zer un fars iran. since. bandar ean za lig ilan iran. diploma in [mathematics](skill) physics. skills. [python](skill) [programming](skill). [tensorflow](skill). [kali linux](skill). [machine learning](skill). [data science](skill). certificates. [python](skill) [programming](skill). april. advanced [python](skill) [programming](skill). january. [machine learning](skill) with [python](skill). speaking level. writing level. listening level. language. english. reading level. speaking level. writing level. listening level. [social network](skill). fard ad. fard ad. fard ad. ferry hi go|
education. sc. sc. [research](skill) interests. [docker](skill) [data science](skill) and [big data](skill). publications. supervisor prof. experiences. recognized. deal with [docker](skill). language knowledge. reading listening. speaking. writing. [computer programming](skill). [machine learning](skill) and [deep learning](skill). familiar and dealing with [linux](skill). familiar and dealing with [matlab](skill). familiar and dealing with [git](skill). [asp net](skill)|
email report. tel. linkedin homepage. linkedin. [research](skill) interests and. skills. [optimization](skill) [deep learning](skill). [biomedical](skill) [signal processing](skill). and [brain computer interface](skill). language. english and persian. [programming](skill) and. [php](skill) [javascript](skill) and [html](skill) [deep learning](skill) toolbox. experience. industrial experience. academic position. education. ph. candidate. in artificial. sc. in [artificial intelligence](skill) [ai](skill). learning. sc. crowded scene. [linux](skill) [red hat](skill) [ubuntu](skill) [centos](skill). [microsoft windows](skill). certificate of attendance. tehran iran. publications. anew method to. iran ian journal of. [medical physics](skill). health and. technology. sparse. journal of health management. informatics. key van pour. minimizing non convex error function. [sensors](skill). sho jae din is. key van pour. anew. genetical gor it hm. journal of biomedical physics and. engineering|
mor var id la leno or. my [github](skill). my email. interests. [machine learning](skill). [computer vision](skill). [medical ai](skill). [medical image analysis](skill). education. credits. june. publication. accepted. using plantar pressure mapping images. accepted. in progress. experience. zeina li. honorand awards. summer. ranked top. selected projects. sept. spatial filtering different images. available project on [gitlab](skill). skills. programmings kills. editors ands oft ware. [anaconda](skill). keil. viva do. [web design](skill). [html](skill) [css](skill). [microprocessor](skill). [git](skill). [github](skill). [gitlab](skill). selected courses. [machine learning](skill). advanced [programming](skill) [java](skill). [digital image processing](skill). online courses. languages kills|
[developer](job_title). [mathematics](skill). [python](skill). english language pros kills. instagram. instagram. tehran. phone. web. masters in [business analytics](skill). university. [python](skill) development. comart in zo mor od ian. supervisor. final project experience. skills. [python](skill) [programming](skill). [machine learning](skill) [algorithms](skill). [linux](skill). [programming](skill) and [data science](skill). peoples code. military service cards er vice. completion card|
personal information. education. sc. sc. in [electrical engineering](skill) [electronics](skill). core units. diploma in [mathematics](skill). [research](skill) interest. [computer vision](skill) and [image processing](skill). [reinforcement learning](skill). publications. submitted. select projects. simulated ina ds environment. [microcontroller](skill) scourse routing robot. work experience. laptop repairs er vice tehran iran. freelancer present. software and hardware diagnosis. [voip](skill) routing. english teacher. [security](skill) equipments installation internship. [teaching assistant](job_title). certifications. advanced [python](skill) course. english [teacher](job_title) training course. nas sir foreign languages center. nas sir foreign languages center. fundamentals of. [net framework](skill) [programming](skill) languages. [technical institute of tehran](university). awards and honors. top among more than participants. affiliations. member of editorial team. central committee member. skills. languages kill. academic module fluent. [programming](skill) skill. computer aided designs oft wares. integrated development environments. hobbies. amateur guitar player. habitual football player. references. sc. supervisor. sc. supervisor|
phone. marital status single. address shiraz. curriculum vitae. educational background. bachelor of [mathematics](skill). diploma. working experience. march until now. trading. september to january. province iran. april to july. [machine learning](skill) [deep learning](skill). [data science](skill). [artificial intelligence](skill). [seaborn](skill). about me. stock market. skills. notes. publications. science era pub. [php](skill) cel article view. comp aper. [html](skill). comp aper maars maars. [html](skill)|
[data scientist](job_title). emails. phone. linkedin. about me. projects. collected at [shiraz university](university). preprocessing data. education. [shiraz university](university) shiraz iran. skills. [python programming](skill). [tensorflow](skill). excellent [communication](skill) skills. proficient in microsoft excel. papers|
kaveh es kanda rim ian doab. education. [university of tehran](university) iran. [research](skill) interests. honors anda wards. sc [student](job_title) in. [teaching](skill) experience. level. mohammad haft javaherian. [research](skill) experience. jun. iran. selected projects. [natural language](skill). word disambiguation tasks. memes seme val task. twitter data. advanced. computational. using [matlab](skill). selected courses. grade. [natural language](skill). grade. grade. advanced [algorithm](skill). grade. advanced. grade. grade. grade. grade. computational. grade. computer and [programming](skill) skills. [operating system](skill) [windows](skill) [linux](skill). language skills. reading listening speaking writing. events and conferences. school on computing. intelligence took part asa participant. september. references. mous avi. ali. institute|
[sql server](skill) querying. institute fad. [data science](skill). [python](skill). boot camp. [python](skill) [flask](skill) boot camp. [sql server](skill) administration. institute fad. data. enthusiastic. skilled. in. [data analysis](skill). [data mining](skill). [python](skill). [power bi](skill). dax. currently. am. technical. lead. at. [business intelligence](skill). improving. products. and. services. for. our. customers. by. using. advanced. analytics. standing. up. bi. tools. creating. and. maintaining. models. and. compelling. new. data. sets. [database developer](job_title). pars royal. tehran iran. august july. tehran iran. since august. tehran iran. bachelor ins oft ware [engineering](skill). tehran iran. mohamad fa lah. dob. marital status. single. [data scientist](job_title). fa lah. mohamad outlook. tehran iran. skills. [python](skill). [microsoft sql server](skill). [data science](skill). [power bi](skill). [data analysis](skill). dax. [database administration](skill). [python](skill) [flask](skill). certificates. profiles umma ry. work experience. education. reading level. writing level. speaking level. listening level. english. mohamad fa lah. competitive game othello. [association rule](skill) mining. [clustering](skill). companies. system. language. [social network](skill). projects. honors|
professional. [data structures](skill). [data modeling](skill). [git](skill) [linux](skill). excellent [communication](skill). [teamwork](skill) and collaboration. make predictions. [algorithm](skill). through text messages. sey ye dj amal hammed an|
tehran iran. skills. technical experience. [data scientist](job_title) present. tehran iran. [student](job_title). [tehran university](university) tehran iran. [student](job_title). education. [university of tehran](university). projects. accurately estimate furnace temperature. crime prediction using boston data. honors. tehran iran. interests. [design](skill) forecasting models. [classification]O(skill) and [clustering](skill) of data. [feature engineering](skill). perform [statistical tests](skill). [data analysis](skill)|
about me. skills. languages. employment history. projects. arrhythmia [classification](skill) tehran. dense net. breast cancer detection tehran. publications. gravitation and [cosmology](skill) [machine learning](skill)|
work experience. [front end developer](job_title) [product design](skill) at. [front end developer](job_title) [product design](skill) at. taxi chimar may. and helped them with [ux design](skill). [front end developer](job_title) [product design](skill) at. [ux designer](job_title) of multiple. [design](skill) multilevel [marketing](skill). new users in months. application. advice on [product design](skill). according tom cire quest. thete am. [css](skill) [html](skill) [adobe](skill). [deep learning](skill) [researcher](job_title). [data analysis](skill). [teacher assistant](job_title) of [ai](skill) courses. [team leader](job_title). [machine learning](skill). [database](skill). [algorithms](skill) area of expertise. during my. statement of purpose present english. native language language proficiency. sc. sc. [programing](skill) languages. and libraries. intermediate level [tensorflow](skill). advanced level [matplotlib](skill). advanced level [pandas](skill). advanced level [python](skill). advanced level. hobbies and. leisure time activities. board games. fun with friends reading. modern literary fiction movies. prestige cinema gaming. st persons hooter mainly music. any genre really. presentation and. documentation. apple numbers. collaborative [prototyping](skill). [microsoft office](skill). [word](skill) [excel](skill) [powerpoint](skill). [deep learning](skill). specialization you. studied the foundational. concept of [neural networks](skill) and [deep learning](skill). [neural networks](skill). and [deep learning](skill). open the [deep learning](skill). black box toun der stand. the processes that drive. performance and. generate good results. systematically. improving [deep neural networks](skill). language how to access. notebooks using [sql](skill). and [python](skill). improving [deep neural networks](skill). learning [data mining](skill). ands tat is tical pattern. recognition. [machine learning](skill) by. [stanford university](university). language according to. [researcher](job_title) in thete am. elites foundation. use [google analytics](skill) in. behavior [google analytics](skill)|
[machine learning engineer](job_title). hamid reza. masha di ran. scholar. google. linkedin. [github](skill). stack over ow. work experience. [machine learning engineer](job_title). tara tech. mash had iran. [manager](job_title) of [research](skill) department. contact. [machine learning engineer](job_title). part [ai](skill). mash had iran. contact. [teaching](skill) experience. laboratory [instructor](job_title). digital laboratory. laboratory [instructor](job_title). logic circuits laboratory. education. medical image. watermarking based on. [deep learning](skill). skills. [deep learning](skill). [machine learning](skill). [python](skill). [tensorflow](skill). [git](skill). [docker](skill). publication. ema mia. bless marka blind. ema mia. gland. networks and handcrafted features. international. reversible image. distortion compensation in wavelet domain. languages. english. per sion. interests. gan. [deep learning](skill). [machine learning](skill). tasks. achievements tasks. thesis|
[back end](skill) [machine learning](skill) [programmer](job_title). learning newt hings is exciting. have. some experience in. [devops](skill). es fah ani ran. work experience. present. remote work. industry. [back end developer](job_title). es fah ani ran. encouraging. workplace. [back end developer](job_title). [microservices](skill). [system analysis](skill). [front end developer](job_title). es fah ani ran. [seo](skill) [web design](skill) and hosting. [front end developer](job_title). education. bachelor of sciences in biotechnology. [university of tehran](university) t eh rani ran. my bachelor project was. about developing better. genetic test. by. analyze. genomic data. with. [machine learning](skill). [algorithms](skill). skills. [python](skill). [tensorflow](skill). [linux](skill). [docker](skill). [asp net core](skill). [entity framework](skill) core. [microservice](skill). [javascript](skill). [clean coding](skill). [solid](skill). [algorithms](skill). languages. english. per sian. interests. [data science](skill). [bioinformatics](skill). [domain driven design](skill). [algorithms](skill). interest. achievements tasks. achievements tasks. achievements tasks. courses|
tehran iran street tehran iran. [python](skill). [machine learning](skill). [deep learning](skill). [java](skill). numerical methods in [linear algebra](skill). [convex optimization](skill). [statistical signal processing](skill). [convolutional neural network](skill). [pattern recognition](skill). [statistical analysis](skill). [random process](skill). detection estimation. [data visualization](skill). [data scientist](job_title). dob. marital status. married. skills. am. [data scientist](job_title). over. five. years. worked. in. data. fusion. and. data. association. projects. using. [machine learning](skill). am. skilled. in. [python programming](skill). and. implementation. of. [algorithms](skill). am. talented. in. math. and. [simulations](skill). bachelor in [electrical engineering](skill). branch [communication](skill). tehran iran. master in [electrical engineering](skill). branch [communication](skill). tehran iran. branch [communication](skill). tehran iran. since. [data scientist](job_title). tehran iran. since september. [data preparation](skill) ands moot hing. [clustering](skill). profiles umma ry. education. work experience. data fusion. data association. [algorithm design](skill). [big data](skill) handling. [java programming](skill). simulations with [matlab](skill). [fuzzy models](skill). projects. reading level. writing level. speaking level. listening level. english. language. [social network](skill)|
contact. address. sat tari highway. boulevard. twelfth alley. phone. email. languages. summary. languages. skill highlights. [machine learning](skill). [deep learning](skill). [artificial intelligence](skill) [data science](skill). [developer](job_title). [programmer](job_title). education. science. [django](skill) [flask](skill). [selenium](skill). certifications. [machine learning](skill) from analytics vidhya. [data science](skill) from alison. [python](skill) from hacker rank. [python](skill) foundation from study section. [excel](skill) from great learning academy|
education. out of. out of. [software engineering](skill). out of. [software technician](job_title). [deep learning](skill). skills. experience. present freelance [developer](job_title). interests. diamond [math](skill) [machine learning](skill). diamond [math](skill) [deep learning](skill). diamond [math](skill) [reinforcement learning](skill). diamond [math](skill) [computer vision](skill). publications. amani decentralized area patrolling for. languages. per sian fluent. turkish first language. references. mobile|
education. academic qua li cations. sc. present. sc. method. notable academic projects. density estimation. [clustering](skill) [unsupervised learning](skill). circle blank [deep learning](skill) projects. tical tests. [simulation](skill) of alane train passage lin. feedback compensator. meta heuristic methods. method. method. semantic priming. prediction of personal morality index. gambling task. bayesian games. games learning inga mes. inga mes. skills. circle blank analytical skills. circle blank computers kills. advanced [verilog](skill). circle blank language. several notable projects areas follow. in te bits channels up to. and [chemical processes](skill)|
amir reza. education. sc. current. attacks. sc. relevant coursework. current. [machine learning](skill) crash course google. current. current. experience. skills. [programming](skill) languages. [python](skill). skilled. experienced. software. skilled. catia. familiar. experienced. writing and presentation. [latex](skill) [microsoft word](skill) [microsoft visio](skill). [microsoft excel](skill). [microsoft powerpoint](skill). selected projects. [matlab](skill). [matlab](skill). strains tressa na lyses oft hey bogie. [matlab](skill). [python](skill)|
marital status. single. military service. served. tehran pars. tehran iran iran. ability. to. implement. [algorithms](skill). from. references. and. [scientific](job_title). articles. and. interested. in. [machine learning](skill). and. [deep learning](skill). [programmer](job_title). tehran iran. since july. realtime facemask detection. we. have. developed. [deep learning](skill). model. for. face. mask. detection. using. [python](skill). and. we. developed. the. face. mask. detector. model. for. detecting. whether. person. is. wearing. mask. or. not. we. have. trained. the. model. using. with. pretrained. mobile net. network. [architecture](skill). [video classification](skill) with [transformer](skill). we. have. developed. hybrid. [transformer](skill) based. models. for. [video classification](skill). that. operate. on. feature. maps. the. data set. consists. of. videos. categorized. into. different. actions. like. cricket. shot. punching. biking. we. use. pretrained. network. for. feature. extraction. histopathologic cancer detection. we. created. an. model. to. identify. metastatic. cancer. in. small. image. patches. taken. from. larger. digital. pathology. scans. the. data. is. slightly. modified. version. of. the. patch camel yon. benchmark. data set. by. using. we. augmented. pictures. for. increase. data set. we. have. developed. deep. learning. model. that. predicts. which. tweets. are. about. real. disasters. and. which. ones. we. use. skip gram. method. for. create. model. we. have. built. [convolutional neural network](skill). model. for. prediction. fluid. mechanics. problems. we. decrease. computational. time. and. increase. accuracy. of. expected. results. we. have. created. network. with. layers. that. predict. stock. behavior. parallelism of. [python](skill). [matlab](skill). [git](skill). [gan](skill). [computer vision](skill). [machine learning](skill). [deep learning](skill). [scrum](skill). reading level. writing level. speaking level. listening level. english. profiles umma ry. work experience. projects. skills. language. [social network](skill). [deep learning](skill) specialization. [ai](skill). [machine learning](skill). institute [stanford university](university). [ai](skill). [ai](skill). master in [aerospace engineering](skill). [toosi university of technology](university). tehran iran. bachelor in [mechanical engineering](skill). [toosi university of technology](university). tehran iran. certificates. education|
linkedin. summary. platforms. education. in [mechatronic engineering](skill). work experiences. accuracy oft hes eme tho ds. extracting the parameters training law. personal projects. automatically with high accuracy. complicated strategy. in the following. this. bot. extracting the parameters training law. certificates and courses. apply [auto encoders](skill) in practice. create basic [tableau](skill) [visualizations](skill). perform [data mining](skill) in [tableau](skill). processing. [neural networks](skill). and [optimization](skill) [deep learning](skill). [ai](skill). applications. [ai](skill). own applications. mic hi gan. courser amar. through optical character recognition ocr. mic hi gan. courser amar. vocational. industrial power technology technical and. vocational. power technology tehran institute. computers kills. interests. [data analysis](skill) [data preprocessing](skill)|
[github](skill). io mobile. [github](skill) [github](skill). education. selected courses. publications. in preparation. skills summary. experience. [machine learning](skill) [researchers](job_title) umm erf all. landmarks. projects. experiments was. lib rosa. december. each data point. honors and awards. sc. events. students umm er. long distance running|
[ai](skill) [researcher](job_title). [deep learning](skill). [ai](skill). [machine learning](skill). [ai](skill) [researcher](job_title). [tensorflow](skill). er|
education. ph. [research](skill) interest. asa ph. links. google. publication. hassan ia. ongoing. misa la. ongoing. hakam in. learning. preprint version. ho mayo un fara. vis uric. pelt one ne. link. link. technicals kills. work experience. improving. interpersonal skills. intelligent system designer [ai](skill). per sian conversation agent. improving. interpersonal skills. [computer engineer](job_title) it. talks and presentation. tehran iran postponed to. sc. projects. classic at ion. additional qualifications and certificates|
mohammad reza. za rei. junior [data science](skill). how tore ach me. home. cell. email. address. mohammad reza. za rei. floors kills. coding languages. skills [python](skill) personal profile. an organization. education. [university of science and technology](university) average. bachelor of industrial. [engineering](skill). [deep learning](skill). [keras](skill) [tensorflow](skill). [machine learning](skill). [image processing](skill). [github](skill). [web scraping](skill) [web crawling](skill). [selenium]. [mysql](skill). [django](skill). [git](skill) familiarity. certificates. details. [neural networks](skill) and [deep learning](skill). structuring [machine learning](skill) projects. [convolutional neural networks](skill). sequence models. [tensorflow](skill) specializa tion. details. [natural language processing](skill) in [tensorflow](skill). sequences times erie sand prediction. details. advanced [computer vision](skill) with [tensorflow](skill). generative [deep learning](skill) with [tensorflow](skill). courses. [machine learning](skill) mastery. top learn. farad ars. dan up. [deep learning](skill). [machine learning](skill) [data mining](skill). [mysql](skill) certificates. details. [python](skill) overview. [python](skill) demonstrations for practice course|
tehran iran. saeed. personal web page. [github](skill). linkedin. elearning. education. bachelors degree. work experience. ow [asp](skill) [project leader](job_title) maryland. july top resent ow [asp](skill) foundation. [security](skill). distributions. junto present ha kin. we collaborate with many. caton et works. cipher. intelligence lab. mentor googles umm er of code. summer of code. with google. dark web crawler. documents andre trie vet hem. [github](skill) gist. salami. skills. [machine learning](skill) and [math](skill). [natural language processing](skill). [deep learning](skill). [information retrieval](skill). [logic](skill) enthusiastic. [linear algebra](skill). [set theory](skill). [software engineer](job_title). [python](skill). [golang](skill). [php](skill). [javascript](skill) intermediate. [bash](skill) expert. [sql](skill) [mysql](skill). [html](skill) [css](skill). [security](skill). protocols and [networking](skill). [cryptography](skill). threat hunting. [wireless network](skill). projects. ow [asp](skill) maryam. ow [asp](skill). page. [github](skill). techniques. the project isa vail able in. [kali](skill) [linux](skill). and. google summer of code. [github](skill). doc term. [github](skill). inter [word embedding](skill). june written in [golang](skill). [github](skill). [github](skill). june written in [python](skill). [github](skill). see course. penetration testing with [golang](skill). see course. [wireless network](skill) attacks. see course. power of ow asp maryam. may ha kin magazine. june ha kin magazine. june pent est magazine. see credential. referees. blank ens hip. ow asp foundation. harold. blank ens hip ow asp. ha kin media. marta. mohammad adel inia. [technical and vocational university](university). adel it vu|
an ou shir van rashidi nia. contact information no. st. tehran iran. phone. email an ou shir van. personal data gender male. place of birth tehran iran. eng. prof. mous avi. eng. kalan tari. [wavelet transform](skill). [vhdl](skill). [microprocessor](skill). eng. courses attended prediction. [statistical pattern recognition](skill). [designing hardware](skill) by [vhdl](skill). [microprocessor](skill). mos avian da. no. percent. accuracy fort ow zing roup. group. malek ne jad. activities gym. soccer. swimming|
sina bag her inez had. ag rea tj obs eek er. sina khan live. coms ina ai. [github](skill). io. objective. [optimization](skill). areas of interest. education. publications. journal of control. capabilities. industrial projects and experience. [engineering](skill) [program manager](job_title). tola un ch. [developer](job_title). query suggestion based onus ers log. [technology consultant](job_title). [technology consultant](job_title). [freelancer](job_title). technical feasibility study. last updated january page. [teaching](skill) experience. [teaching assistant](job_title) ins tat is tics. certificates. [deep learning](skill) specialization july. issued by [deep learning](skill). [neural networks](skill) and [deep learning](skill). [convolutional neural networks](skill). sequence models. fundamentals of [reinforcement learning](skill). sample based learning methods. issued by [deep learning](skill). technical [communication](skill) skills. programming languages [python](skill). natural languages english per sian. achievements. sc. sc. entrance exam in management. personal attributes. last updated january page|
education. [robotics](skill). related courses. work. [graphical models](skill) [computer vision](skill). recognition. oriented design [database management](skill) [data structure](skill) [algorithm design](skill). [engineering](skill). [azad university central tehran branch](university). tehran iran. date of graduation december. undergraduate thesis. process. career objective. determinations dedication and hard. work. amir nas simi. [artificial intelligence programmer](job_title). linkedin. behind page. experienced in [python](skill) [programming](skill). libraries. [solidity](skill) language good understanding of. [cryptography](skill). experienced with modern [machine learning](skill). [image processing](skill) [computer vision](skill). [algorithms](skill) and [deep learning](skill) models. working knowledge of coracle and [mysql](skill) database. excellent interpersonal and [communication](skill). skills. effective technical instructions and. presentations kills. extensively within [java](skill) programs. skills qualification. references. professional experience. predictive maintenance [ai](skill) eng. toga ina. models. charts. personal interests. interests including new ideas and. experiences water polo and hiking. furnished upon request. masters thesis abstract. low dimensional representation. methods have been proposed. well. results forme tri clear ning. deep localized metric learning. finally anew. criteria. the samples in both. multiple cameras from two angles. finally according tother ich accuracy. samples|
[quantitative analysis](skill). personal information. location tehran iran. mobile. email mei sam. linkedin. presentation. advanced [mathematical techniques](skill). experience. present [quantitative analysis](skill) charisma iran. market. derive tes pricing. [data scientist](job_title) mell at bank iran. iran. report magazine. banking industry. my main task. [sentimental analysis](skill) [bert](skill). [customers segmentation](skill) [clustering](skill). customer rating. [customers segmentation](skill). [recommendation engine](skill). web scraping. education. ai. google cloud. computer and programmings kills. data vis. package and platforms. machine. webs carping. developing. languages|
email mo ka. address tehran. education bachelor of [software](skill). phone. age. [clean code](skill). experience. [front end developer](job_title). tehran iran. since december march. company basal am. [front end developer](job_title). tehran iran. [front end developer](job_title). [javascript](skill) [html](skill) [css](skill) [bootstrap](skill). tehran iran. skills. [react](skill). [ant design](skill). [figma](skill) [redux](skill). [style component](skill). [git](skill) [git flow](skill). [html](skill) [html](skill). [javascript](skill). [responsive design](skill) [bootstrap](skill). [postman](skill)|
[front end developer](job_title). born. marital status married. mobile. address iran tehran. military services tatus exempt. email kamran. about. two years of experience. skills. [javascript](skill) es. [typescript](skill). [redux](skill) [redux thunk](skill). [git](skill) [git flow](skill). languages. english. tor ki. email. kamran. linkedin. work experience. system harmonic. [front end developer](job_title). dai ere. [front end developer](job_title). [front end developer](job_title). [freelancer](job_title). [front end developer](job_title). activity summary. education. bachelors. degree. projects. [front end developer](job_title). [front end developer](job_title). sabas er vice. [front end developer](job_title). [github](skill). stack over ow. personal website. day ere. of elevators and escalators. civil share. for locals. [front end developer](job_title). online. certifications|
work experience. education and training. languages kills. mother tongues per sian. other languages english. digitals kills. [front end](skill). [ui](skill) frameworks. [bootstrap](skill) [tailwind](skill) [css](skill) [material ui](skill). source controller. [git](skill) [github](skill) [gitlab](skill). [ui design](skill). nationality iran ian. gender male. contact. linkedin. sari iran. [front end developer](job_title). [software engineering](skill). commo oner geek realtime chat app|
language per sian english|
about me. working with [react](skill). work experience. education and training. digitals kills. [front end](skill). [design](skill). [adobe photoshop](skill) [adobe xd](skill) [figma](skill). [ui](skill) frameworks. [bootstap](skill) [material ui](skill) [tailwind](skill) [css](skill). [source controller](skill). [git](skill) [github](skill) [gitlab](skill). [back end](skill). [node](skill). hes ami. date of birth. nationality iran ian. gender male. contact. linkedin. hes ami. sari iran. sari iran. website features. technologies frameworks. sari iran. [front end developer](job_title). [intern web developer](job_title). [computer engineering](skill). languages kills. mother tongues per sian. other languages. english. listening. breading. production. interaction. [react](skill)|
[front end developer](job_title). tehran iran. linkedin. mm ad reza. [github](skill). commo ham madre za al. skills. [javascript](skill). [redux](skill). [recoil](skill). [typescript](skill). [html](skill). [css](skill). [tailwind](skill) [css](skill). [git](skill). [sass](skill). languages. english. per sian. education. present. tehran iran. personal projects. twitter clone. on this project you. source. live. discord clone. source|
[software engineer](job_title) [web developer](job_title). profile. technologies and tools. code. languages. english fluent details. military services tatus. end of service. birthdate. skills. [html](skill) [css](skill). [javascript](skill). [sass](skill). [git](skill). [python](skill). [sql](skill). [linux](skill) experience. vakil jo. while as [react](skill) [developer](job_title). contact. iran tehran. educations. resigned. resigned|
curriculum vitae. [agile development](skill) [team manager](job_title). sc. [cognitive science](skill) [research](skill). born. cellphone. linkedin. [razor pages](skill). [asp net](skill) [mvc](skill) [core](skill). [agile web programming](skill) with [asp](skill). professional experience. tehran iran. [razor pages](skill) [mvc](skill) [core](skill) now. rad in. tehran iran. [windows application programmer](job_title). tele rik [wpf](skill) [ui](skill) and reporting. tehran iran. [project manager](job_title) [programmer](job_title). tehran iran. eniac tech. tehran iran. [scrum masters](job_title) up port [manager](job_title). [agile](skill) [scrum](skill) [kanban](skill). iran nara. tehran iran. sparrow bankings oft ware. iran ian customs. tehran iran. national project. ikea it global production. educt us. [computer teacher](job_title). uni net. malms wed en. [web programmer](job_title). [robot programming](skill) [research](skill) in [artificial intelligence](skill). [web developer](job_title). [html](skill) front page [dream weaver](skill). [computer teacher](job_title). [web design](skill) with [html](skill). commentor. education. [java programming](skill). itil. [cognitive science](skill) [research](skill) program. industrial [programming](skill). malms wed en. [electronic engineering](skill). nor rare als tock holms wed en. iran za mint eh rani ran. technicals kills. [razor pages](skill). [asp net](skill) [mvc](skill) [core](skill). [entity framework](skill) [linq](skill). tele rik [wpf](skill) [ui](skill) andre porting. [sharepoint](skill) administration installation. customization. [mobile programming](skill) [asp net](skill) [mvc](skill). programming languages [asp](skill). [visual basic](skill). [oracle](skill) agents. databases [microsoft sql server](skill). front page. access [sharepoint](skill). languages. english fluent. swedish fluent. courses. [artificial intelligence](skill) for robots. [artificial intelligence](skill). [java programming](skill). [compiler programming](skill). modern [operating systems](skill). [human computer interaction](skill). [logic programming](skill) with [prolog](skill). function [programming](skill) withs che me. formal languages anda ut oma tes. [electronic engineering](skill). [mathematics](skill). [cognitive science](skill)|
team player punctual. [computer engineering](skill). [software](skill). ma zan daran. [cofounder](job_title). big follower. [cofounder](job_title). [developer](job_title). [science and technology](university) of. ma zan daran. [back end developer](job_title). [senior developer](job_title). alireza. [mobile clients](skill) oft ware [architecture](skill). [mobile client](skill) deployment. [windows server administrator](job_title). [mobile clients](skill) oft ware [architecture](skill). [product owner](job_title). [windows server administrator](job_title). [software architechture](skill). [multilayer architecture](skill). [linux](skill) [server administrator](skill). [microservice](skill). [redis](skill). [web scraping](skill). [back end developer](job_title). fara nam. [asp](skill). [python](skill). [java](skill). [agile software development](skill). [restful web service](skill). [git](skill). [back end developer](job_title). [encryption](skill). [linq](skill) to [sql](skill). [oauth](skill). [socket programming](skill). [web scraping](skill). [redis](skill). [microservice](skill). [android developer](job_title). [debian](skill) based [linux](skill). [administrating windows server](skill). [raspberry pi](skill). [mvc](skill) [php](skill). [react native](skill). [deep learning](skill). [machine learning](skill). [application architecture](skill). [javascript](skill). [web service](skill). tehran. electro habib is tore website. online store. harmony cafe. freelancing. alireza es fah ani. ref actor all|
[back end developer](job_title). mobile. work experience. present. [nexus repository](skill). homings tart up. [python](skill) [beautiful soup](skill). caring startup. this. dongles hoppings er vices. administration ands up port panels. [net core](skill). development of crm panel [asp](skill). [net](skill) [mvc](skill). [java](skill). [java](skill). side projects. devices. [net](skill) [windows form](skill). mechanism and execute them. [net](skill) windows service. education. tools skills methodologies. [windows](skill) [linux](skill) [docker](skill). [net core](skill). [net framework](skill) [git](skill). [java](skill) [python](skill) [nexus repository](skill) [manager](job_title). familiar with. [elk stack](skill) [nginx](skill)|
amir mohammad agha pour. [web development](skill). console app desktop app web app. ia ma [back end](skill) and [front end](skill). [html](skill) [css](skill). [sqlite](skill) [asp](skill). united hardworking and hardworking boy. nis hing university. [engineering](skill). personal info. date of birth. address. marital status. single. email. mobile number. telephone. [software engineer](job_title). [web development](skill). duration tehran. east. city tehran. bachelor. [software engineer](job_title). among the top university students. languages. native. experience. education. awards. skills|
contact. con. skills. [mysql](skill) [postgresql](skill). [git](skill) [gitlab](skill). [linux](skill). [weka](skill). [asp](skill). [net](skill) [angular](skill) [react](skill). [django](skill). now. experience. ia is. [web developer](job_title). it rc. trainee. working on [openstack](skill) software. [university of tehran](university) database lab. sc. final project. education. sc. [software engineering](skill). [university of tehran](university). sc. [software engineering](skill). interests. relevant coursework. [neural networks](skill) and [deep learning](skill). [statistical inference](skill). subject toran dom variation. certificates. [network](skill). [android](skill)|
[english teacher](job_title) ina language institute. yu|
phone. email zahra. education. additional. well language proficiency. familiar with [crypto currency](skill). full familiarity with [search engines](skill). [planning](skill) and [time management](skill). [research](skill) skills. familiar with [wordpress](skill). organizational skills. english professional working proficiency|
[content specialist](job_title). per sian [english translator](job_title). language proficiency. english advanced. de uts cha. skills. [content writing](skill). [link building](skill). [producing and finding keywords](skill). [joomla](skill). can va. [photoshop](skill). work experience. recently. [web content writer](job_title). [content writer](job_title). tappers ia. [content writer](job_title). achievements. andrea ch more customers. profile male married. company looking for new. opportunities. location tehran mas oo di ye. current position [content specialist](job_title) [translator](job_title). mobile. email. saman. what canid of or. you. english specialized [translator](job_title). your websites traffic and google. rank [writing](skill) and producing suitable. [seo](skill) friendly content according to. and cellphone friendly rules. other companies such as tap. my writings on googles first page. ia maty ours er vice if youa re. looking fora [content specialist](job_title) who. makes the [wordpress](skill) all green in. achievements. [windows](skill). [office tools](skill). [word](skill) [excel](skill) [powerpoint](skill). internet. personal growth. related relevant fields such as. [digital marketing](skill). english stories. travel. education. its kills. interests|
[content specialist](job_title). military service educational exemption. marital status married. day of birth. profiles umma ry. same field. group asa [freelance writer](job_title). [content marketing](skill). work experience. since july. tehran iran. educating new employees in content team. freelance [content writer](job_title) at dam oon. tehran iran. tehran iran. junior [content specialist](job_title). tehran iran. tehran iran. skills. [copywriting](skill) [text editing](skill). [google ad words](skill) [adobe premiere pro](skill). [adobe photoshop](skill). educational records. bachelor of [english literature](skill). diploma of [math](skill). language. english. reading level. writing level. listening level. speaking level|
summary. years experience. ability. honesty iso ne. situations. [marketing manager](job_title). skills. [leadership](skill). targeting. [analytics](skill). [strategic planning](skill). publicity [campaign](skill). [advertising](skill) planning physical digital. [performance optimization](skill). [data analysis](skill). [site management](skill). [social network management](skill). [seo](skill). [compilation of clips](skill). [edit images](skill). generate and enter data. [performance evaluation](skill). prepare are port. security education personal information. year of birth. gender male. networks. ya shar bani hashem. vulnerabilities. [advanced robotics](skill) degree. schools in tehran province. [mathematics](skill) and physics diploma. contact. [site management](skill). [ads on sites](skill). [cyberspace management](skill). [content production](skill). categories oft ar gets. [advertising consultant](job_title) [planning](skill) and target ting. [competitors analysis](skill). [site management](skill). [application management](skill). [ads](skill). networking. increase conversion rate exchanger ate. [digital marketing](skill) [planning](skill). [market capitalization analysis](skill). virtual [targeted ads](skill). [communication](skill) with clients. [social network management](skill). [marketing plan](skill). [periodic reporting analysis](skill). [process optimization](skill). [content team management](skill) and [design](skill) [team manager](job_title) [digital marketing](skill). [planning](skill) and managing virtual activities. social channels and pages. [website design](skill) planning. [graphics](skill). [programmer](job_title). football bart arf rom july to july. course conference workshop. [site management](skill). [data collection](skill). [technical analysis](skill). thinking about product presentation. [instagram admin](skill). [marketing plan](skill). short term and longterm activities. [digital marketing](skill). [telemarketing](skill). [facet of ace marketing](skill). requirements. costes tim at ion. content releases che dule. organizational color. organizational chart. [graphic](skill). [web design](skill)|
shah rams ha ki bibs hah ram. sha ki big mail. skills summary. [marketing](skill) and [branding](skill). [market research](skill). specialists hah rams ha ki bi. educational history. [strategy](skill) [management](skill). [management](skill). work experience. present|
[salesman](job_title) [strategist](job_title)|
skills. [python](skill). time. [django](skill). [django rest framework](skill). [database](skill). [postgresql](skill) [mysql](skill). [web design](skill). [wordpress](skill). professional [wordpress](skill) work. other skills. education. bachelors degree [computer software](skill). info. age. programmer freelance now. languages english per sian. [github](skill). iran tehran. nima. objective. technologies andi also want to. experience new challenges and|
[robotic](skill). [software. developer](job_title). since january. [back end developer](job_title). accelerator. april july. [back end developer](job_title). os tova. studio. july april. [back end developer](job_title) and [devops](skill). [developer](job_title). dob. machines. make automation for creating data set. snap food. [microservices](skill) architectural and. [programming](skill). language. [python](skill). [javascript](skill). [django](skill). [django rest framework](skill). [express](skill). [sql](skill). [mysql](skill). [mongo](skill). [redis](skill). [docker](skill). [docker](skill). [docker](skill) compass. [nginx](skill). [git](skill). [microservice](skill). architecture. [selenium](skill). mocha. pro. work experience. skills. conte mo. [back end developer](job_title). al om edi cal. tehran iran. since may. bachelor. in. computer. engineering. branch. [software](skill). university. since. [research](skill). for. [decision making](skill). service. for. robot. [research](skill). for. the. best. way. for. path. planing. in. the. environment. considering. the. tasks. hosting. site. for. september. make. website. with. [django](skill). and. template. engine. kang roo. for. os tova. studio. july. make. [back end](skill). with. and. kangaroo. is. the. parental. control. application. its. for. manage. children. with. parents. link. kangaroo. land. reading level. writing level. speaking level. listening level. english. conte mo. conte mo. education. research. projects. language. [social network](skill). contact. windows. application. january. make. windows. form. application. with. and. [sql](skill). [database](skill). for. manage. organization. contact. lap vann. for. lap vann. april. make. [back end](skill). with. [django rest. framework](skill). for. lap vann. [advertising](skill). application. link. lap vann|
tab ass omen te za mi. [back end developer](job_title). tab ass om. en te za mig mail. birth year marital status single. about me. education. [azad university](university). [computer engineering](skill). tehran. far zane gan. [mathematics](skill). tehran. links. linkedin tab ass omen te za mi. [github](skill) tab ass omen te za mi. skills. [python](skill). [functional programming](skill). [scripting](skill) [json](skill) [logging](skill). [marshmallow](skill). [django](skill). [django rest framework](skill). [rest permissions](skill). [custom admin panel](skill). testing. [web design](skill). [html](skill) [css](skill) [flex box](skill). [responsive design](skill) [bootstrap](skill). [ajax](skill) [wordpress](skill) [javascript](skill). other. [git](skill) [redis](skill) [celery](skill) experience. assignment [management](skill). [architecture](skill). freelance. the site. sections by. developed on [django](skill) framework. september. june. courses. coding boot camp [python](skill) [django](skill) hours. tehran|
arvin sade ghi. contact. my personal website. linkedin profile. my [github](skill). highlights. structured. consistent. fast learner. friendly. summary. entrepreneurial space. skills. [mobile application](skill) development [flutter](skill). test [crawling](skill) framework [selenium](skill). [computer vision](skill). experience. [full stack freelancer](job_title) to. clients ide framework. [teacher assistance](job_title) present. ped ram fall. mem arian if all. mirta her if all. mirta her if all. issue. issue. education. tehran iran|
[python](skill) [django](skill) [developer](job_title). [scrum](skill) [agile](skill) [django rest framework](skill). [django](skill). [python](skill). [django rest framework](skill). recipe app. [django](skill) [css](skill) [html](skill) [python](skill) [javascript](skill). [django rest framework](skill) [docker](skill) [python](skill)|
alireza ger ami. ger ami. shiraz iran. linkedin. git hub. comal ire zag era mi. about. highly. organized. and. detail oriented. [computer engineering](skill). [student](job_title). with. years. experience. in. [python](skill). from. the. university. of. with. relevant. knowledge. in. restful. [sql](skill). [databases](skill). and. excellent. skill. and. teamwork. ability. looking fora remote. entry level. position. asa. [django](skill) [back end developer](job_title). experience. [front end](skill) [back end developer](job_title). [freelancer](job_title). shiraz fars iran. months. [django](skill). and. [javascript](skill). unavailable. [django](skill). for. [metatrader expert](job_title). shiraz fars iran. months. an. online. [python](skill). training. course. from. elementary. to. intermediate. in. manner. under. the. supervision. [student](job_title) intern. fars science and technology park. shiraz fars iran. months. restful. by. [django](skill). [flask](skill). base on. [docker](skill). [teacher assistants](job_title). education. sc [computer engineering](skill). present years. skills. knowledgeable. [python](skill) [django](skill) [web scraping](skill). familiar. [flask](skill) [javascript](skill) [html](skill) [css](skill) [postgresql](skill). languages. english intermediate. soft skills. [time management](skill). volunteer experience. years. years. challenges faced by the [student](job_title) body|
tehran iran. education. thesis grade excellent. under supervision of prof. awards honors. association competition iran. [research](skill) interests. experiences. [university tehran](university) iran. young adult kara ji ran. [university tehran](university) iran. [university tehran](university) iran. kara ji ran. publications. in preparation. mes gar if. published. iran. iran. workshops seminars. certificated online conference on. december. [university of mashhad](university) mash had. iran. academic projects. skills languages. [programming](skill) languages. [python](skill). [matplotlib](skill) [seaborn](skill) [tensorflow](skill). designing plotting. [latex](skill) [microsoft office](skill). [adobe photoshop](skill) [adobe premier](skill). reference managing. bib tex endnote. others. [linux](skill) [git](skill) computational. [software](skill). [wolfram](skill). [astronomical](skill). softwares. viewer alma. languages. english fluent. volunteer works memberships. [university tehran](university) iran. [university tehran](university) iran. hobbies. others. references. prof. ria zig mail. prof. mohammad ali ganja li. ganja lithe ory|
[research](skill) and working interest. [deep learning](skill). [machine learning](skill). [pattern recognition](skill). [game theory](skill). education. [electrical engineering](skill) control. out of. out of. [electrical engineering](skill) control. out of. out of. iran. [electrical engineering](skill) control. out of. out of. honors awards. exam for universities. ranked in the top. sc degree of [electrical engineering](skill) control. publication. journal papers. no. [research](skill) experiences. freedom. sc. market via game theoretic approaches. pricing. ara abi. [nonlinear systems](skill). ar van. sc. working experiences. parse hins tit ute. no. [teaching](skill) experiences. graduate undergraduate. [game theory](skill). [pattern recognition](skill). computers kills. languages kills. english fluent. arabic familiar. courses. selected courses. [stochastic process](skill) [pattern recognition](skill). [game theory](skill) [neural network](skill). distributed [optimization](skill) [smart grid](skill). extracurricular activities interests. movies anime climbing. football. no|
mashhad iran. linkedin. summary. companies and customers. organizations. professional. experience. daya soft iran. [python developer](job_title) full timer emote. january april. andre liability. customer complaints by. lang unity germany. [python developer](job_title) full time remote. remotely. freelance. [python developer](job_title). march january. prepared [software](skill) for customer needs. websites. projects. [architect](job_title). cocreator. deed [ai](skill). [developer](job_title). creator. skills. [python](skill) [django](skill). [aws](skill) ecs. [gnu](skill) [linux](skill). [design patterns](skill). [sql](skill) [postgresql](skill). [git](skill). [docker](skill). [scrum](skill). [teamwork](skill). [testing](skill)|
"""



eval_text = """
rast khan eh yahoo. [microsoft excel](skill). [microsoft powerpoint](skill). [microsoft word](skill). [python](skill). [matlab](skill). job vision|
sina bag her inez had. ag rea tj obs eek er. sina khan live. coms. [github](skill). io. objective. [optimization](skill). areas of interest. education. publications. journal of control. capabilities. industrial projects and experience. [engineering](skill) [program manager](job_title). tola un ch. [developer](job_title). query suggestion based onus ers log. [technology consultant](job_title). [technology consultant](job_title). [freelancer](job_title). technical feasibility study. last updated january page. [teaching](skill) experience. [teaching assistant](job_title) in [operations research](skill). [teaching assistant](job_title) in [statistics](skill). certificates. issued by [deep learning](skill). [neural networks](skill) and [deep learning](skill). structuring [machine learning](skill) projects. [convolutional neural networks](skill). sequence models. fundamentals of [reinforcement learning](skill). sample based learning methods. issued by [deep learning](skill). [convolutional neural networks](skill) in [tensorflow](skill). sequences [time series](skill) and prediction. technical communication skills. programming languages [python](skill). natural languages english per sian. achievements. sc. entrance exam in computer engineering. sc. entrance exam in management. personal attributes. last updated january page|
cafe bazaar. smartest. android len. bit. yousef pour. [product designer](job_title). work experience. skills. [research](skill). sos kills. languages. linkedin. fara bi accelerator. landing page. they were developing the site. also coll abo rang with. no vin. tech accelerator. iran. [designer](job_title). summer ad versing. goals. devs. dates in calendar. tech company. summer autumn erp. stories. also [designing sketches](skill) ow diagrams. also is tarted learning. developers. autumn winter ad versing. carpet. [graphics](skill) user ow concepts ketches. [persona and journey maps](skill). spring. autumn. winter. logic [programming](skill). summer. [product design](skill). [product design](skill). [ui ux designer](job_title). [ui ux designer](job_title). [ui ux designer intern](job_title). team. we have. grooming sessions for new features. bonuses and trading. wed id. and had interviews with persons. sprints oft he [design](skill) team and dev. za re last art up. developers. brand. on the [ux researches](job_title) wed id. delivering the product tot he dev. team. thema in. with [rahnama college](university). we asa teams tarted by. we run. we. provided context map and blueprint map. [making personas](skill). yet|
be hance linkedin matin ej ava di. [product designer](job_title). education [ui ux designer](job_title). jan may. [software engineering](skill). [software engineering](skill). [design guidelines](skill). experiences for [users flows](skill) and generating multiple ideas. [testing](skill). flows. specific feedback on my designs. that form new products and features. [design](skill). licenses. skills. [design](skill). tools. [figma](skill). [scrum](skill) [user flows](skill). [adobe photoshop](skill). [time management](skill) [usability test](skill). [zeplin](skill) [design thinking](skill). [sketch](skill). ideation [wireframe](skill). [adobe xd](skill). english intermediate level|
experience. present. to. increasing the sale rate by percent. others. ail. education. academic education. department of engineering. department of science. related education. google [ux design](skill) courser. arts courser. languages kills. listening reading. writing. tools. [adobe xd](skill). [adobe photoshop](skill). [figma](skill). [moz](skill). [google analytics](skill). [adobe illustrator](skill). marvel. skills. [wireframing](skill). [rapid prototyping](skill). [interaction design](skill). leadership|
[social media marketing](skill). [digital marketing](skill). current city tehran. mob ina good ar zig mail. linkedin. ba education. until may. proficiency. [advertising](skill). [campaign](skill). [business model canvas](skill). skills. [microsoft office](skill). [wordpress](skill) [google analytics](skill). [designing](skill). [social media analysis](skill). proposal [design](skill) [research](skill). about me. courses. new product development lunch. fundamentals of [digital marketing](skill). [digital marketing](skill) foundations. linkedin learning. linkedin learning. courser awork experience. january until june|
skills. [python](skill). time. [django](skill). [django rest framework](skill). [database](skill). [postgresql](skill) [mysql](skill). [web design](skill). [wordpress](skill). professional [wordpress](skill) work. other skills. education. bachelors degree [computer software](skill). info. age. programmer freelance now. languages english per sian. [github](skill). iran tehran. nima. objective. technologies andi also want to. experience new challenges and|
[robotic](skill). [software. developer](job_title). since january. [back end developer](job_title). accelerator. april july. [back end developer](job_title). os tova. studio. july april. [back end developer](job_title) and [devops](skill). [developer](job_title). dob. machines. make automation for creating data set. snap food. [microservices](skill) architectural and. [programming](skill). language. [python](skill). [javascript](skill). [django](skill). [django rest framework](skill). [express](skill). [sql](skill). [mysql](skill). [mongo](skill). [redis](skill). [docker](skill). [docker](skill). [docker](skill) compass. [nginx](skill). [git](skill). [microservice](skill). architecture. [selenium](skill). mocha. pro. work experience. skills. conte mo. [back end developer](job_title). al om edi cal. tehran iran. since may. bachelor. in. computer. engineering. branch. [software](skill). university. since. [research](skill). for. [decision making](skill). service. for. robot. [research](skill). for. the. best. way. for. path. planing. in. the. environment. considering. the. tasks. hosting. site. for. september. make. website. with. [django](skill). and. template. engine. kang roo. for. os tova. studio. july. make. [back end](skill). with. and. kangaroo. is. the. parental. control. application. its. for. manage. children. with. parents. link. kangaroo. land. reading level. writing level. speaking level. listening level. english. conte mo. conte mo. education. research. projects. language. [social network](skill). contact. windows. application. january. make. windows. form. application. with. and. [sql](skill). [database](skill). for. manage. organization. contact. lap vann. for. lap vann. april. make. [back end](skill). with. [django rest. framework](skill). for. lap vann. [advertising](skill). application. link. lap vann|
areas of [research](skill) interest. [machine learning](skill). [signal processing](skill). [big data analysis](skill). [software engineering](skill). [robotics](skill). [control theory](skill). languages. english advanced and fluent. total. listening. reading. writing. speaking. contact. phone. email. education. bachelors [electrical engineering](skill). in. the last years. graduated in may. diploma [physics](skill) and [mathematics](skill). work experience. some curricular courses. [linear algebra](skill). [computer programming](skill). what languages this is too. general. [signals](skill) systems. [microprocessor](skill) and. [computer structure](skill). [artificial intelligence](skill) and. [bio calculations](skill). extracurricular courses. introduction tow aves and vibrations. principles. online course offered by. open courseware website. [python](skill) course. certificate of completion. with bands core out of. [matlab](skill) course. course offered bys hah id. certificate of completion. with bands core out of. course offered bys hah id. certificate of completion. with bands core out of. [django](skill) course. self learned via. youtube. [machine learning](skill) with [python](skill). course. self learned via. youtube. projects. shazam project via [python](skill). [python](skill). search fora specific one. skills. programmings kills. [python](skill) advanced. [javascript](skill) intermediate. [version controller](skill). [github](skill) intermediate. [sql](skill) intermediate. [matlab](skill) intermediate. [css](skill) intermediate. [html](skill) intermediate. [wordpress](skill) intermediate. [fpga](skill) beginner. [php](skill) beginner. [simulation](skill) programs. proteus. [teamwork](skill) skills. collaboration. [leadership](skill). [critical thinking](skill). publications. in preparations due fall. honors anda wards. top. top|
[software developer](job_title). and. [engineer](job_title). with. years. experience. mostly. and. actively. in. [back end](skill). besides. basic. [devops](skill). and. basic. [front end](skill). experiences. main. technologies. and. [architectures](skill). that. professional. at. them. [python](skill). [django](skill). [docker compose](skill). [microservices](skill). [postgresql](skill). services. [redis](skill). [elk](skill). [react](skill). [nginx](skill). pipelines. [linux](skill). and. [bash](skill). also. familiar. with. [typescript](skill). [javascript](skill). [node](skill). [express](skill). [kafka](skill). [sql server](skill). [windows server](skill). and. [java](skill). contact. better. aly. khor rama bad lores tani ran. [software engineer](job_title) fulls tack. dob. marital status. single. military service. not served. technology. khor rama bad lores tani ran. university. tehran iran. realty na. spain. profiles umma ry. education. associate. in. computer. science. and. [programming](skill). bachelor. in. [computer science](skill). and. [programming](skill). work experience. senior. [software engineer](job_title). full time. remote. december june. [architecture](skill). musers goal. [network](skill) mm users goal. musers goal. [software engineer](job_title). part time. remote. peached. [python](skill). [django](skill). [postgresql](skill). [git](skill) [git flow](skill) [github](skill) flow. [celery](skill). [flask](skill). [jira](skill) [confluence](skill) [bitbucket](skill). [linux](skill) [bash](skill). [nginx](skill). [docker compose](skill). [microservices](skill). [architecture](skill). [redis](skill). [serverless](skill). services. [html](skill) [css](skill). andi sheva ran. rah bord. armin. zas. tehran iran. [lorestan university](university). cert. khor rama bad lores tani ran. po nisha. tehran iran. skills. [senior software engineer](job_title). december december. [software developer](job_title). [software developer](job_title). [freelancer](job_title). language. per sian. reading level. speaking level. writing level. listening level. english. reading level. speaking level. writing level. listening level. [jenkins](skill). [azure](skill). [devops](skill). [react](skill). [sql server](skill). [node](skill). [java](skill). institute. institute. lynda institute. lynda. institute. for. realty na. users. mm. goal. [social network](skill). to. connect. all. realtors. and. real. estate. agents. of. and. goal. for. all. around. the. world. and. migrate. data. from. and. rets. gain. experiences. in. [social networks](skill). load. and. stress. tests. realtime. chat. systems. [ai](skill). services. and. data. modeling. data. ranking. platforms. and. [recommendation systems](skill). payment. services. stripe. and. link. realty feed. for. ara. iran. in obi. tech. users. handling. many. services. including. captive. portal. with. governmental. users. credential. management. with. and. premium. memberships. panels. managing. drivers. buses. with. realtime. [monitoring](skill). and. reporting. services. working. mainly. with. gps. devices. middle ware. for. remote. configuration. and. update. providers. and. iran ian. payment. services. advertisement. panels. for. party. business. customers. link. in obi. tech. [social network](skill). certificates. advanced. [python](skill). course. series. [devops](skill). roadmap. courses. [javascript](skill). [front end](skill). roadmap. courses. [android development](skill). with. [java](skill). projects. realty feed. september. in obi. [back end](skill). server. for. in obi. tech. dub ai. canada. not. finished. due. to. lack. of. funds. geolocation based. [social network](skill). to. track. user. pings. on. the. map. having. feed. for. users. posts. and. shares. filtering. the. data. based. on. the. users. distance. with. support. for. corona virus. status. updates. and. pings. [back end](skill). developed. by. [python](skill) [django](skill). [database](skill). [postgresql](skill). [redis](skill). [elasticsearch](skill). [front end](skill). [angular](skill). and. [ionic](skill). link. in obi. tech. for. iran. lores tan. an. enterprise. learning. management. system. for. [lorestan university](university). students. teachers. panel. to. add. courses. with. dynamic. lectures. system. video. voice. document. on. live. streaming. solutions. parental. panel. for. basing. project. in. schools. certification. issuance. for. students. on. courses. optional. payment. for. selling. certifications. courses. or. lectures. notification. services. with. [advertisement](skill). services. with. [push notifications](skill). auto. generation. of. learning. roadmaps. based. on. teacher. levels. courses. categories. or. students. picks. for. british. client. peach land. team. users. shop. [management](skill). system. providing. store. and. [management](skill). panel. for. different. user. types. implementing. payment. engine. provider. integrated. with. stripe. and. pay pal. link. peached. for. kei van. trading. [strategy](skill). for. interactive. brokers. implemented. watch list. generation. based. on. prepared. data. models. implemented. [strategy](skill). in. [sql server](skill). procedures. and. triggers. working. with. packages. implemented. in. [python](skill). middle ware. library. mixture. of. [event driven](skill). and. concurrent. paradigms. many. integration. load. and. stress. tests. for. performance. and. accuracy. to. be. launched. on. options. market. implemented. realtime. logging. and. reporting. services. manipulation. and. on. data sets. link. interactive brokers. [github](skill). [html](skill). in [oop](skill) in. peached. april. trading. strategy. bot. april|
amir husain za rei. linkedin. summary. crucial roles like. writing [clean code](skill). have ane ye on best practices. experience. [back end developer](job_title). may present year months. create new features. ref actor code. develop pure [python](skill) module. integrate third party components. education. skills. development [git](skill) [redis](skill) [react](skill). amir husain za rei page|
[back end developer](job_title). bandar ean za lig ilan. ecommerce web app. buy products. [django](skill) packages. news web app. articles and news. [postgresql](skill) [docker](skill) and [django](skill) packages. [message queue](skill). blog web app. articles. [django](skill) packages. experience. [python](skill). [oop](skill). [design patterns](skill). [django](skill). [django rest](skill). [sql alchemy](skill). [alembic](skill). [postgresql](skill). [celery](skill). [redis](skill). [elasticsearch](skill). [docker](skill). [git](skill). [scrum](skill). [linux](skill). english intermediate. [software engineering](skill). [azad university](university) bandar ean za li. working asa [back end developer](job_title). technologies and [teamwork](skill). summary. skills. languages. education|
[junior back end developer](job_title). [docker](skill) and [flask](skill). build and develop new ideas. tehran iran. linkedin. work experience. [junior back end developer](job_title). present. tehran iran. on coding and debugging. bid specialist. mona construction and facilities. tehran iran. contract. bid specialist. tehran iran. deadlines andi dent if ying tasks. education. masters degree [computer software engineering](skill). tehran iran. bachelors degree applied [mathematics](skill). tehran iran. skills. [mathematics](skill). [python](skill) [programming](skill) language. [sql alchemy](skill). [postgresql](skill). [git](skill). [linux](skill). [docker](skill). [flask](skill). [microsoft sql server](skill). certificates. present. [python](skill) [programming](skill) language. professional training center. [university of tehran](university). languages. per sian. english. interests. reading. theater. photography. travelling. achievements tasks. achievements tasks. achievements tasks|
full stack. personal info. cell. email. linkedin. linkedin. fars iran careers umma ry. [full stack developer](job_title). logistic problems. developing [android](skill) and web. dashboard using [flutter](skill). family members. using [docker](skill) [docker swarm](skill). gathering metrics with [prometheus](skill). creating aping checker dashboard rah pa. ia ma [full stack developer](job_title) with. experience in [back end](skill). [android](skill) [front end](skill) and. [devops](skill) technologies. and work with teams. that can level my skills. up. birthdate. [developer](job_title). creating anew scrawler anda chat. client using [electron](skill) casted away. pj. five needs. su. lo. po. english fluency. reading. listening. speaking. [django](skill) application development. working on different [django](skill) based. projects. developing ash op if yapp. working on [python](skill) and [golang](skill). services. january top resent. [android](skill) technologies. native [devops](skill) technologies. [devops](skill). [dokcer](skill). [docker swarm](skill). [nginx](skill) [back end](skill) technologies. [python](skill). [django](skill). [tornado](skill). [concurrent programming](skill). [selenium](skill). [peewee](skill). [celery](skill). [golang](skill). [gorilla](skill). [gorm](skill) soft skills. room. view binding. [alarm manager](skill) work. manager. [navigation component](skill). background foreground. services. not native. [flutter](skill) [front end](skill) technologies. [front end](skill) frameworks. [angular](skill). [flutter](skill). [front end](skill) designing. [html](skill). [css](skill). non relational. [redis](skill). relational. [mysql](skill) other technologies. [git](skill). [serialization](skill). [google protocol buffers](skill). others. [algorithems](skill). [data structures](skill). [architectures](skill). cafe bazaar crawler. [gitlab](skill) repo. [django](skill). [docker](skill). [docker compose](skill). [postgresql](skill) teaches tan. [github](skill) repo. [navigation component](skill). jabin [android](skill) app. room. view binding. workman ager. [navigation component](skill). foreground services. not available. [golang](skill). will be available shortly. [django](skill). [prometheus](skill)|
"""


raw_text = "|".join([remove_stopwords(x) for x in raw_text.split("|")])

raw_text = re.sub("[.]", "", raw_text)
raw_text = re.sub("\s+", " ", raw_text)

dm = NERDataMaker(raw_text.split("|"))


eval_text = "|".join([remove_stopwords(x) for x in eval_text.split("|")])

eval_text = re.sub("[.]", "", eval_text)
eval_text = re.sub("\s+", " ", eval_text)

ev = NERDataMaker(eval_text.split("|"))


total examples = 107


In [ ]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(dm.unique_entities), id2label=dm.id2label, label2id=dm.label2id)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=60,
    weight_decay=0.01,
)

train_ds = dm.as_hf_dataset(tokenizer=tokenizer)
eval_ds = ev.as_hf_dataset(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 107
  Num Epochs = 60
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 420
  Number of trainable parameters = 66368263
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__c

Epoch,Training Loss,Validation Loss
1,No log,1.110520
2,No log,0.999335
3,No log,0.843700
4,No log,0.724773
5,No log,0.636087
6,No log,0.565041
7,No log,0.508713
8,No log,0.461848
9,No log,0.439273
10,No log,0.415359


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens. If id, ner_tags, tokens are not expected by `DistilBertForTokenClassification.forward`,  you can sa

TrainOutput(global_step=420, training_loss=0.17250929332914808, metrics={'train_runtime': 7130.6299, 'train_samples_per_second': 0.9, 'train_steps_per_second': 0.059, 'total_flos': 334121153467662.0, 'train_loss': 0.17250929332914808, 'epoch': 60.0})

In [ ]:
from transformers import pipeline
pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
test_resume = """front end developer. professional experience. front end developer. app. components. tehran iran. skills. bootstrap. css. sass less. \
                 windows forms. xamarin html. javascript. [material ui](skill) ant design. wpf. education. bachelor of hardware engineering. \
                 tehran iran. courses. languages. persian english"""

test_resume = remove_stopwords(test_resume)
test_resume = re.sub("[.]", "", test_resume)
test_resume = re.sub("\s+", " ", test_resume)

pipe(test_resume)

[{'entity_group': 'job_title',
  'score': 0.9960647,
  'word': 'front end developer',
  'start': 0,
  'end': 19},
 {'entity_group': 'job_title',
  'score': 0.9959903,
  'word': 'front end developer',
  'start': 44,
  'end': 63},
 {'entity_group': 'skill',
  'score': 0.9718268,
  'word': 'bootstrap',
  'start': 98,
  'end': 107},
 {'entity_group': 'skill',
  'score': 0.9959165,
  'word': 'css',
  'start': 108,
  'end': 111},
 {'entity_group': 'skill',
  'score': 0.9903821,
  'word': 'sass',
  'start': 112,
  'end': 116},
 {'entity_group': 'skill',
  'score': 0.9964759,
  'word': 'windows',
  'start': 122,
  'end': 129},
 {'entity_group': 'skill',
  'score': 0.98082066,
  'word': 'xamarin',
  'start': 136,
  'end': 143},
 {'entity_group': 'skill',
  'score': 0.99816906,
  'word': 'html',
  'start': 144,
  'end': 148},
 {'entity_group': 'skill',
  'score': 0.99649596,
  'word': 'javascript',
  'start': 149,
  'end': 159},
 {'entity_group': 'skill',
  'score': 0.96569884,
  'word': 'materi